In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install numpy 
!pip install pandas
!pip install sklearn
!pip install json
!pip install fast_ml

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
     |████████████████████████████████| 42 kB 786 kB/s 


In [ ]:
import numpy as np #Für Matrizen
import pandas as pd #Für Dataframes
import re
from typing import List, Any, Set, Dict, Tuple, Optional
from ast import literal_eval
import json # load json module
from fast_ml.model_development import train_valid_test_split
import sys

In [ ]:
excel_path = '/content/drive/MyDrive/train_data.xlsx'
sheet_name = "labeled_dataset3"

In [ ]:
df = pd.read_excel(excel_path, sheet_name, index_col=None, dtype={'Label': np.int64})
df = df.astype({'Label': str})
print(df)
print(len(df))

                                                   Text      Label
0     Withdrawal arrhythmia  Injury of intercostal a...         -1
1     Typhlolithiasis  Injury of ascending right col...         -1
2     Severe manic bipolar I disorder without psycho...         -1
3     Poisoning by sawfly larvae  Hemorrhagic bronch...         -1
4     Miscarriage with laceration of cervix  Chronic...         -1
...                                                 ...        ...
2468                Stress echocardiography (procedure)  816996009
2469                Stress echocardiography (procedure)  816996009
2470                            Stress echocardiography  816996009
2471  Regular participation in cardiological special...  816996009
2472  Performing or assisting with stress echocardio...  816996009

[2473 rows x 2 columns]
2473


In [ ]:
df['Label'].value_counts()

363704007    59
387713003    19
405815000    17
441619002    17
-1           13
             ..
71861002      1
424226004     1
65818007      1
61160002      1
785804000     1
Name: Label, Length: 410, dtype: int64

In [ ]:
dict_name = "l_dict_m4"
json_dir = f'/content/drive/MyDrive/{dict_name}.json'

In [ ]:
with open(json_dir) as jsonFile:
    labels_Input = json.load(jsonFile)
    jsonFile.close()
print(max(labels_Input), min(labels_Input))

9724000 -1


In [ ]:
labels_Output = {}
for k, v in labels_Input.items():
    labels_Output[v["index"]] = {
            "name": v["name"],
            "synonym": v["synonym"],
            "code": k,
            "index": v["index"]
            } 
print(labels_Output)
print(len(labels_Output))

{0: {'name': 'none of the trained concepts', 'synonym': [], 'code': '-1', 'index': 0}, 1: {'name': 'Excision of mediastinal tumor (procedure)', 'synonym': "['Excision of mediastinal tumor', 'Excision of mediastinal tumour']", 'code': '2407009', 'index': 1}, 2: {'name': 'Stripping (procedure)', 'synonym': "['Stripping']", 'code': '2677003', 'index': 2}, 3: {'name': 'Laryngeal structure (body structure)', 'synonym': "['Larynx', 'Laryngeal structure', 'Larynx structure']", 'code': '4596009', 'index': 3}, 4: {'name': 'Three dimensional ultrasound imaging of heart (procedure)', 'synonym': "['Three dimensional ultrasound imaging of heart', 'Three dimensional echocardiography']", 'code': '5216004', 'index': 4}, 5: {'name': 'Mastoid antrotomy (procedure)', 'synonym': "['Mastoid antrotomy', 'Transmastoid antrotomy']", 'code': '5892005', 'index': 5}, 6: {'name': 'Chondrectomy of semilunar cartilage of knee (procedure)', 'synonym': "['Chondrectomy of semilunar cartilage of knee', 'Arthrectomy of 

In [ ]:
label_code = []
code_fails = []
for index, row in df.iterrows():
    if row["Label"] in labels_Input:
        label_code.append(labels_Input[row["Label"]]["index"])
    else:
        label_code.append(labels_Input["-1"]["index"])

df['label_code'] = label_code
print(df)

                                                   Text      Label  label_code
0     Withdrawal arrhythmia  Injury of intercostal a...         -1           0
1     Typhlolithiasis  Injury of ascending right col...         -1           0
2     Severe manic bipolar I disorder without psycho...         -1           0
3     Poisoning by sawfly larvae  Hemorrhagic bronch...         -1           0
4     Miscarriage with laceration of cervix  Chronic...         -1           0
...                                                 ...        ...         ...
2468                Stress echocardiography (procedure)  816996009         368
2469                Stress echocardiography (procedure)  816996009         368
2470                            Stress echocardiography  816996009         368
2471  Regular participation in cardiological special...  816996009         368
2472  Performing or assisting with stress echocardio...  816996009         368

[2473 rows x 3 columns]


In [ ]:
df['label_code'].value_counts()

0      217
317     19
348     17
160     11
133     11
      ... 
107      4
338      4
311      4
286      4
339      4
Name: label_code, Length: 369, dtype: int64

In [ ]:
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df, target = 'label_code', 
                                                                            train_size=0.7, valid_size=0.15, test_size=0.15)



df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train.index.values, 'data_type'] = 'train'
df.loc[X_valid.index.values, 'data_type'] = 'val'
df.loc[X_test.index.values, 'data_type'] = 'test'

df.groupby(['Label', 'label_code', 'data_type']).count()

Text
Label    label_code data_type      
-1       0          test          1
                    train        12
10200004 8          test          2
                    train         3
                    val           1
...                             ...
91101005 169        val           1
91381003 170        test          3
                    train         3
9724000  7          test          1
                    train         5

[914 rows x 1 columns]

In [ ]:
count=0
for k,v in labels_Input.items():
    print(k , v)
    df.loc[df[df.Label ==k].first_valid_index(), 'data_type'] = 'train'
    count+=1
    print(f'{count} / {len(labels_Input)}')
    
    

df.groupby(['Label', 'label_code', 'data_type']).count()

In [ ]:
excel_path = '/content/drive/MyDrive/train_ready_data_m3_v1.xlsx'
sheet_name = "train_ready_dataset_m3_v1"

In [ ]:
with pd.ExcelWriter(excel_path) as writer:  
    df.to_excel(writer, sheet_name=sheet_name, index=False)